In [1]:
import requests
import pandas as pd
from splinter import Browser
from bs4 import BeautifulSoup as bs
from webdriver_manager.chrome import ChromeDriverManager

In [2]:
# Setup splinter

executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)



====== WebDriver manager ======
Current google-chrome version is 99.0.4844
Get LATEST chromedriver version for 99.0.4844 google-chrome
Trying to download new driver from https://chromedriver.storage.googleapis.com/99.0.4844.51/chromedriver_win32.zip
Driver has been saved in cache [C:\Users\ccrom\.wdm\drivers\chromedriver\win32\99.0.4844.51]


## NASA MARS NEWS

In [ ]:
# URL to visit
url = 'https://redplanetscience.com/'
browser.visit(url)

# Scrape the latest News Title & Paragraph Text 
html = browser.html
soup = bs(html, 'html.parser')

news_title = soup.find('div', class_='content_title').text

news_p = soup.find('div', class_='article_teaser_body').text

print(f"Title: {news_title}")
print(f"Article: {news_p}")

# # # Quit browser once scraping is complete
# browser.quit()


## JPL Mars Space Image

In [ ]:
# Visit new site to find image jpg. for feature image
url = 'https://spaceimages-mars.com'
browser.visit(url)

# Scrape site for featured image jpg.
html = browser.html
soup = bs(html, 'html.parser')

results = soup.find_all('div', class_= 'floating_text_area')

for result in results:
    link = result.a['href']
    featured_image_url = f'https://spaceimages-mars.com/{link}'

    print(featured_image_url)
    
# browser.links.find_by_partial_text('FULL IMAGE').click()

## Mars Facts

In [ ]:
# Use pandas to scrape table from url
url = 'https://galaxyfacts-mars.com'

# read in tables, and print them
tables = pd.read_html(url)
tables

In [ ]:
# Index into list to see the tables we have: Mars & Earth Comparison
tables[0]

In [ ]:
# Index into second list item where we have Mars facts only (I prefer this one)
tables[1]

In [ ]:
# Save desired table to Variable df 
df = tables[1]
df

In [ ]:
# Saving desired table to html and not including the header or index
df.to_html('marsfact_table.html', header = False, index = False, classes= 'table')

## Mars Hemispheres

In [3]:
# Visit new site to find image jpg. for feature image
url = 'https://marshemispheres.com/'
browser.visit(url)

In [4]:
html = browser.html
soup = bs(html, 'html.parser')

results = soup.find_all('div', class_='description')

In [5]:
title_list = []

for result in results:
    title_txt = result.find('h3').text
    title = title_txt.split('Enhanced')[0]
    title_list.append(title)

In [6]:
title_list

['Cerberus Hemisphere ',
 'Schiaparelli Hemisphere ',
 'Syrtis Major Hemisphere ',
 'Valles Marineris Hemisphere ']

In [ ]:
title_list

In [ ]:
title_list[0]

In [ ]:
for index, value in enumerate(title_list):
    if index == 4:
        break
        

In [7]:
imgurl_list = []

hemisphere_image_urls = []

for index, value in enumerate(title_list):
    
    if index == 4:
        break
    else:
        url = 'https://marshemispheres.com/'
        browser.visit(url)  
        browser.links.find_by_partial_text(value).click()

        html = browser.html
        soup = bs(html, 'html.parser')
        results = soup.find_all('div', class_='downloads')

        for result in results:
            u_list = result.find('ul')
            l_list = u_list.find('li')
            article = l_list.find('a')
            link = article['href']
            img_url = f'https://marshemispheres.com/{link}'

            img_dict = {
                'title' : value,
                'img_url' : img_url
            }

            hemisphere_image_urls.append(img_dict)
            
            index+=1


print(hemisphere_image_urls)
            
            


[{'title': 'Cerberus Hemisphere ', 'img_url': 'https://marshemispheres.com/images/full.jpg'}, {'title': 'Schiaparelli Hemisphere ', 'img_url': 'https://marshemispheres.com/images/schiaparelli_enhanced-full.jpg'}, {'title': 'Syrtis Major Hemisphere ', 'img_url': 'https://marshemispheres.com/images/syrtis_major_enhanced-full.jpg'}, {'title': 'Valles Marineris Hemisphere ', 'img_url': 'https://marshemispheres.com/images/valles_marineris_enhanced-full.jpg'}]


In [ ]:
hemisphere_image_urls

In [ ]:
# fruit_dictionary = dict(zip(fruits, prices))
# print(fruit_dictionary)

In [ ]:
hemisphere_image_urls = dict(zip(title_list,url_list))

print(hemisphere_image_urls)

In [ ]:





titles = [
    'title_1',
    'title_2',
    'title_3'
]

authors = [
    'author_1',
    'author_2',
    'author_3'
    
]

In [ ]:
book = dict(zip(titles, authors))

print(book)



In [ ]:
book_list = []

book_list.append(book)

print(book_list)

In [ ]:
url = 'https://marshemispheres.com/cerberus.html'
browser.visit(url)
html = browser.html
soup = bs(html, 'html.parser')

In [ ]:
results = soup.find_all('div', class_='downloads')
print(results)

In [ ]:
for result in results:
    u_list = result.find('ul')
    l_list = u_list.find('li')
    link = l_list.find('a')
    href = link['href']
    imgurl_list.append(href)

In [ ]:
imgurl_list

In [ ]:
# Visit new site to find image jpg. for feature image
url = 'https://marshemispheres.com/'
browser.visit(url)

# Scrape site for featured image jpg.
# html = browser.html
# soup = bs(html, 'html.parser')

# results = soup.find_all('div', class_= 'description')
html = browser.html
soup = bs(html, 'html.parser')

results = soup.find_all('div', class_='description')

for result in results:
    title = result.find('h3').text
    print(title)
    
    browser.links.find_by_partial_text('Hemisphere').click()
    

    
#     title = result.find('h3')
#     print(title)
    
    
    
#     for result in results:
#         title = soup.find('article', class_='itemLink product-item').text
#         print(Title)
#     browser.links.find_by_partial_text('Hemisphere').click()
    
#     soup = bs(html, 'html.parser')
# #     results = soup.find_all('div', class_='downloads')
        
#         for result in results:
#             title = soup.find('div', class_='content_title').text
            
    


In [8]:
# # Quit browser once scraping is complete
browser.quit()